In [ ]:
# How do payment methods influence customer purchasing patterns and category preferences?
import pandas as pd

sellers = r"./data/olist_sellers_dataset.csv"
sellers = pd.read_csv(sellers)

category_name_translation = r"./data/product_category_name_translation.csv"
category_name_translation = pd.read_csv(category_name_translation)

orders = r"./data/olist_orders_dataset.csv"
orders = pd.read_csv(orders)

order_items = r"./data/olist_order_items_dataset.csv"
order_items = pd.read_csv(order_items)

payments = r"./data/olist_order_payments_dataset.csv"
payments= pd.read_csv(payments)

order_reviews = r"./data/olist_order_reviews_dataset.csv"
order_reviews = pd.read_csv(order_reviews)

geolocation = r"./data/olist_geolocation_dataset.csv"
geolocation = pd.read_csv(geolocation)

customers = r"./data/olist_customers_dataset.csv"
customers = pd.read_csv(customers)

products = r"./data/olist_products_dataset.csv"
products = pd.read_csv(products)

# Convert dates in orders
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'], dayfirst=True)

# Merge datasets
full_df = (
    orders.merge(payments, on='order_id')
    .merge(order_items, on='order_id')
    .merge(products, on='product_id')
    .merge(customers, on='customer_id')
    # Filter by delivered orders
    .query("order_status == 'delivered'")
)

# Calculate meaningful metrics
payment_behavior = (
    full_df.groupby(['customer_unique_id', 'payment_type'], as_index=False)
    .agg(
        total_spend=('price', 'sum'),
        order_count=('order_id', 'nunique'),
        favorite_category=('product_category_name', lambda x: x.mode()[0] if len(x.mode()) > 0 else None)
    )
)

# Identify preferred payment method (highest spend)
preferred_payment = (
    payment_behavior.sort_values('total_spend', ascending=False)
    .drop_duplicates('customer_unique_id')
    [['customer_unique_id', 'payment_type', 'total_spend', 'favorite_category']]
)

# Merge with original data to get all transactions
enhanced_df = full_df.merge(
    preferred_payment[['customer_unique_id', 'payment_type']],
    on='customer_unique_id',
    suffixes=('', '_preferred')
)

# Calculate actionable metrics by preferred payment
results = (
    enhanced_df.groupby('payment_type_preferred')
    .agg(
 customers=('customer_unique_id', 'nunique'),
        total_orders=('order_id', 'count'),
        total_revenue=('price', 'sum'),
        avg_order_value=('price', 'mean'),
        avg_orders_per_customer=('customer_unique_id', lambda x: x.count() / x.nunique()),  # Correct,
        top_categories=('product_category_name', lambda x: x.value_counts().index[0]),
        pct_repeat_customers=('customer_unique_id', lambda x: x.duplicated(keep=False).mean())
    )
    .sort_values('avg_order_value', ascending=False)
)

# Calculate avg order value
results['avg_order_value'] = results['total_revenue'] / results['total_orders']

results['avg_order_value'] = results['avg_order_value'].round(2)
results['pct_repeat_customers'] = (results['pct_repeat_customers'] * 100).round(1)

print("Customer Behavior by Preferred Payment Method:")
print(results)

# Calculate % of customers who bought from their top category by payment type
top_category_analysis = (
    enhanced_df.groupby(['payment_type_preferred', 'product_category_name'])
    .agg(customers=('customer_unique_id', 'nunique'))
    .reset_index()
)

# Get the top category for each payment type
top_categories = results['top_categories'].to_dict()

# Calculate percentages
top_category_percentages = []
for payment_type, top_cat in top_categories.items():
    total_customers = results.loc[payment_type, 'customers']
    top_cat_customers = top_category_analysis[
        (top_category_analysis['payment_type_preferred'] == payment_type) &
        (top_category_analysis['product_category_name'] == top_cat)
    ]['customers'].values[0]
    
    percentage = (top_cat_customers / total_customers) * 100
    top_category_percentages.append({
        'payment_type': payment_type,
        'top_category': top_cat,
        'pct_customers_in_top_category': round(percentage, 1)
    })

# Create and display results
top_category_df = pd.DataFrame(top_category_percentages)
print("\nPercentage of Customers Who Bought From Top Category:")
print(top_category_df[['payment_type', 'top_category', 'pct_customers_in_top_category']])

Customer Behavior by Preferred Payment Method:
                        customers  total_orders  total_revenue  \
payment_type_preferred                                           
credit_card                 70663         84518    10623078.61   
debit_card                   1446          1654      178543.21   
voucher                      2674          6547      685375.84   
boleto                      18574         22316     2326831.05   

                        avg_order_value  avg_orders_per_customer  \
payment_type_preferred                                             
credit_card                      125.69                 1.196071   
debit_card                       107.95                 1.143845   
voucher                          104.69                 2.448392   
boleto                           104.27                 1.201464   

                                top_categories  pct_repeat_customers  
payment_type_preferred                                                
credi

In [3]:
# Does delivery performance affect customer satisfaction (review score)?)

# Convert date columns into the correct format (yyyy/m/d)
date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
orders[date_cols] = orders[date_cols].apply(lambda x: pd.to_datetime(x, dayfirst=True, errors='coerce'))

# Calculate delivery time metrics
orders['actual_delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders['estimated_delivery_time'] = (orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']).dt.days
orders['delivery_time_difference'] = orders['actual_delivery_time'] - orders['estimated_delivery_time']

# Merge with review scores
df = orders.merge(order_reviews[['order_id', 'review_score']], on='order_id', how='inner')

# Filter valid deliveries
df = df.dropna(subset=['actual_delivery_time', 'review_score'])

# Bucket delivery performance
bins = [-float('inf'), -3, 0, 3, 7, float('inf')]
labels = ['Too Early (>3d)', 'Slightly Early (0-3d)', 'Ideal (1-3d late)', 'Late (4-7d)', 'Very Late (8+d)']
df['delivery_performance'] = pd.cut(df['delivery_time_difference'],
                                   bins=bins,
                                   labels=labels)

# Advanced Analysis - Review Score Distribution
print("\nReview Score Distribution by Delivery Performance:")
performance_stats = df.groupby('delivery_performance')['review_score']\
                      .agg(['mean', 'median', 'count', lambda x: (x < 3).mean() * 100])
performance_stats.columns = ['mean_score', 'median_score', 'order_count', 'low_rating_pct']
print(performance_stats.sort_values('mean_score', ascending=False))

# Additional Insight: Early Delivery Problem
too_early = df[df['delivery_performance'] == 'Too Early (>3d)']
print(f"\nOrders arriving >3 days early have {too_early['review_score'].mean():.1f} avg rating "
      f"({(too_early['review_score'] < 3).mean()*100:.1f}% low ratings)")

# First get all orders by seller for baseline
seller_totals = (df.merge(order_items, on='order_id')
                 .groupby('seller_id')['order_id']
                 .count()
                 .rename('total_orders'))

# Then analyze late orders
late_sellers = (df[df['delivery_performance'].isin(['Late (4-7d)', 'Very Late (8+d)'])]
                .merge(order_items, on='order_id')
                .groupby('seller_id')
                .agg(
                    late_orders=('order_id', 'count'),
                    avg_review_score=('review_score', 'mean'),
                    avg_days_late=('delivery_time_difference', 'mean')
                )
                .merge(seller_totals, left_index=True, right_index=True)
                .assign(
                    late_pct=lambda x: (x['late_orders'] / x['total_orders'] * 100).round(1),
                    impact_score=lambda x: x['late_pct'] * x['late_orders']  # Prioritize high volume offenders
                )
                .sort_values('impact_score', ascending=False)
                .head(10)
                .reset_index()
                [['seller_id', 'late_orders', 'total_orders', 'late_pct', 
                  'avg_review_score', 'avg_days_late']]
)

print("\nTop 10 Problematic Sellers by Late Delivery Impact:")
print(late_sellers.to_string(float_format="%.1f", index=False))



Review Score Distribution by Delivery Performance:
                       mean_score  median_score  order_count  low_rating_pct
delivery_performance                                                        
Too Early (>3d)          4.301290           5.0        84636        9.134411
Slightly Early (0-3d)    4.120044           5.0         4540       11.519824
Ideal (1-3d late)        3.613413           4.0         2341       23.280649
Late (4-7d)              2.231868           1.0         1751       63.906339
Very Late (8+d)          1.717567           1.0         3091       78.680039

Orders arriving >3 days early have 4.3 avg rating (9.1% low ratings)

Top 10 Problematic Sellers by Late Delivery Impact:
                       seller_id  late_orders  total_orders  late_pct  avg_review_score  avg_days_late
4a3ca9315b744ce9f8e9374361493884          160          1949       8.2               2.1           13.5
06a2c3af7b3aee5d69171b0e14f0ee87           59           396      14.9           

  performance_stats = df.groupby('delivery_performance')['review_score']\
